In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.metrics import mean_squared_error

In [33]:
housing = pd.DataFrame(pd.read_csv('Housing.csv'))
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [34]:
m = len(housing)

In [35]:
#Splitting the Data into Training and Testing Sets
from sklearn.model_selection import train_test_split

#We specify this so that the train and test data set always have the same rows, respectively
np.random.seed(0)
df_train, df_test = train_test_split(housing, train_size = 0.8, test_size = 0.2)
df_train.shape, df_test.shape

((436, 13), (109, 13))

In [36]:
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'price']
df_Newtrain = df_train[num_vars]
df_Newtest = df_test[num_vars]
df_Newtrain.head()

,area,bedrooms,bathrooms,stories,parking,price
542,3620,2,1,1,0,1750000
496,4000,2,1,1,0,2695000
484,3040,2,1,1,0,2870000
507,3600,2,1,1,0,2590000
252,9860,3,1,1,0,4515000


In [37]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler, StandardScaler
m_scaler = MinMaxScaler()
s_scaler = StandardScaler()
#MinMax First
df_Newtrain[num_vars] = s_scaler.fit_transform(df_Newtrain[num_vars])
df_Newtest[num_vars] = s_scaler.fit_transform(df_Newtest[num_vars])

In [38]:
Y_Normtrain = df_Newtrain.pop('price')
X_Normtrain = df_Newtrain.copy()
Y_Normtest = df_Newtest.pop('price')
X_Normtest = df_Newtest.copy()

In [39]:
Y = Y_Normtrain.values[:]
Y_test = Y_Normtest.values[:]

In [40]:
X1 = df_Newtrain.values[:, 0] #Area values
X2 = df_Newtrain.values[:, 1] #Bedroom values
X3 = df_Newtrain.values[:, 2] #Bathroom values
X4 = df_Newtrain.values[:, 3] #Stories values
X5 = df_Newtrain.values[:, 4] #Parking values

X1_test = df_Newtest.values[:, 0] #Area values
X2_test = df_Newtest.values[:, 1] #Bedroom values
X3_test = df_Newtest.values[:, 2] #Bathroom values
X4_test = df_Newtest.values[:, 3] #Stories values
X5_test = df_Newtest.values[:, 4] #Parking values

In [41]:
m_train = len(Y_Normtrain)
m_test = len(Y_Normtest)
m_train, m_test

(436, 109)

In [42]:
X_0 = np.ones((m_train, 1))
X_0_test = np.ones((m_test, 1))

In [43]:
#Reshape function used to convert X from 1D to 2D array of m x 1
X_1 = X1.reshape(m_train, 1)
X_2 = X2.reshape(m_train, 1)
X_3 = X3.reshape(m_train, 1)
X_4 = X4.reshape(m_train, 1)
X_5 = X5.reshape(m_train, 1)

X_1_test = X1_test.reshape(m_test, 1)
X_2_test = X2_test.reshape(m_test, 1)
X_3_test = X3_test.reshape(m_test, 1)
X_4_test = X4_test.reshape(m_test, 1)
X_5_test = X5_test.reshape(m_test, 1)

In [44]:
#hstack() function from numpy to have X_0 to X_5 in one final Matrix
X = np.hstack((X_0, X_1, X_2, X_3, X_4, X_5))
X_test = np.hstack((X_0_test, X_1_test, X_2_test, X_3_test, X_4_test, X_5_test))

In [45]:
theta = np.zeros(6)
theta

array([0., 0., 0., 0., 0., 0.])

In [46]:
def model(X1, X2, X3, X4, X5, W1, W2, W3, W4, W5, b):
    return W5*X5 + W4*X4 + W3*X3 + W2*X2 + W1*X1 + b

In [47]:
#Predicted vs Actual
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [49]:
#Loss Calculation
W1 = torch.ones(())
W2 = torch.ones(())
W3 = torch.ones(())
W4 = torch.ones(())
W5 = torch.ones(())
b = torch.zeros(())
t_p = model(X1, X2, X3, X4, X5, W1, W2, W3, W4, W5, b)

In [50]:
loss = loss_fn(t_p, Y)
loss

tensor(6.0517, dtype=torch.float64)

In [51]:
delta = 0.1
pos_rate_of_change_w = (loss_fn(model(X1, X2, X3, X4, X5, W1+delta, W2+delta, W3+delta, W4+delta, W5+delta, b), Y))
neg_rate_of_change_w = loss_fn(model(X1, X2, X3, X4, X5, W1-delta, W2-delta, W3-delta, W4-delta, W5-delta, b), Y)
loss_rate_of_change_w = pos_rate_of_change_w - neg_rate_of_change_w
loss_rate_of_change_w = loss_rate_of_change_w/(2.0*delta)
loss_rate_of_change_w


tensor(14.5259, dtype=torch.float64)

In [52]:
learning_rate = 1e-2
W1 = W1 - learning_rate * loss_rate_of_change_w
W2 = W2 - learning_rate * loss_rate_of_change_w
W3 = W3 - learning_rate * loss_rate_of_change_w
W4 = W4 - learning_rate * loss_rate_of_change_w
W5 = W5 - learning_rate * loss_rate_of_change_w

In [53]:
loss_rate_of_change_b = (loss_fn(model(X1, X2, X3, X4, X5, W1, W2, W3, W4, W5, b+delta), Y) - loss_fn(model(X1, X2, X3, X4, X5, W1, W2, W3, W4, W5, b-delta), Y)) / (2.0 * delta)
loss_rate_of_change_b
b = b - learning_rate * loss_rate_of_change_b

In [54]:
def dloss_fn(t_p, t_c):
    dsq_diffs = 2 * (t_p - t_c) / t_p.size(0)
    return dsq_diffs

In [55]:
def dmodel_dw5(X5, W5, b):
    return X5

In [56]:
def dmodel_dw4(X4, W4, b):
    return X4

In [57]:
def dmodel_dw3(X3, W3, b):
    return X3

In [58]:
def dmodel_dw2(X2, W2, b):
    return X2

In [59]:
def dmodel_dw1(X1, W1, b):
    return X1

In [60]:
def dmodel_db(X1, X2, X3, X4, X5, W1, W2, W3, W4, W5, b):
    return 1.0

In [61]:
#Grad Descent
def grad_fn(X1, X2, X3, X4, X5, Y, t_p, W1, W2, W3, W4, W5, b):
    dloss_dtp = dloss_fn(t_p, Y)
    dloss_dw5 = dloss_dtp * dmodel_dw5(X5, W5, b)
    dloss_dw4 = dloss_dtp * dmodel_dw4(X4, W4, b)
    dloss_dw3 = dloss_dtp * dmodel_dw3(X3, W3, b)
    dloss_dw2 = dloss_dtp * dmodel_dw2(X2, W2, b)
    dloss_dw1 = dloss_dtp * dmodel_dw1(X1, W1, b)
    dloss_db = dloss_dtp * dmodel_db(X1, X2, X3, X4, X5, W1, W2, W3, W4, W5, b)
    return torch.stack([dloss_dw5.sum(), dloss_dw4.sum(), dloss_dw3.sum(), dloss_dw2.sum(), dloss_dw1.sum(), dloss_db.sum()])

In [68]:
def training_loop(X1, X2, X3, X4, X5, n_epochs, learning_rate, params, Y):
    for epoch in range(1, n_epochs + 1):
        W1, W2, W3, W4, W5, b = params
        
        t_p = model(X1, X2, X3, X4, X5, W1, W2, W3, W4, W5, b)
        loss = loss_fn(t_p, Y)
        grad = grad_fn(X1, X2, X3, X4, X5, Y, t_p, W1, W2, W3, W4, W5, b)
        
        params = params - learning_rate * grad
        
        if (epoch == 1) or (epoch % 500 == 0):
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            print('MSE: ', mean_squared_error(Y, t_p))
            print(' ')
    return params

In [71]:
training_loop(X1, X2, X3, X4, X5, n_epochs = 5000, learning_rate = .1, params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0]), Y=Y)

Epoch 1, Loss 6.051744
MSE:  6.051743994927176
 
Epoch 500, Loss 31798652701215372803411839896379500261553655164960768.000000
MSE:  3.1798652701215373e+52
 
Epoch 1000, Loss 66447879863059804620739533827196205346466414842636639932255723329183033602961805980500860371173559425302528.000000
MSE:  6.6447879863059805e+106
 
Epoch 1500, Loss 138852465232559283587873036878683254170087387503682622649826004031096299923613391420045346321749948847300974210821942496739767382083366362205837784927892083310592.000000
MSE:  1.3885246523255926e+161
 
Epoch 2000, Loss 290152329026798446479831728525799263787177309252690669219286512379176178278850868488301058404723962972155467909225563817372900174901715686442883364526919175478699573952208961590591518773042410750442178778450081349632.000000
MSE:  2.9015232902679845e+215
 
Epoch 2500, Loss 60631529947034984868803490306880676036823088074817936923046968798535285438847542759745567270684468772204048110181542003792091651173901886547942072658093306663871758282125

tensor([-4.8160e+270,  6.4864e+269, -7.5757e+268, -5.2442e+269,  4.7649e+270,
        -4.5801e+254], dtype=torch.float64)

In [72]:
training_loop(X1, X2, X3, X4, X5, n_epochs = 5000, learning_rate = .01, params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0]), Y=Y)

Epoch 1, Loss 6.051744
MSE:  6.051743994927176
 
Epoch 500, Loss 11731.571059
MSE:  11731.571058686628
 
Epoch 1000, Loss 6076953235.491063
MSE:  6076953235.491063
 
Epoch 1500, Loss 3352154898781116.500000
MSE:  3352154898781116.5
 
Epoch 2000, Loss 1871756928109572521984.000000
MSE:  1.8717569281095725e+21
 
Epoch 2500, Loss 1047500267155873829870370816.000000
MSE:  1.0475002671558738e+27
 
Epoch 3000, Loss 586460391441711310241041219584000.000000
MSE:  5.864603914417113e+32
 
Epoch 3500, Loss 328364505819491900864750013827168337920.000000
MSE:  3.283645058194919e+38
 
Epoch 4000, Loss 183856831649306375120600711635191588993368064.000000
MSE:  1.8385683164930638e+44
 
Epoch 4500, Loss 102944807327006448847779776422626491437115615739904.000000
MSE:  1.0294480732700647e+50
 
Epoch 5000, Loss 57640709982026289646845963731612358098336236653628096512.000000
MSE:  5.764070998202629e+55
 


tensor([-6.6519e+27,  8.9701e+26, -1.0467e+26, -7.2542e+26,  6.5813e+27,
        -6.9377e+11], dtype=torch.float64)

In [73]:
training_loop(X1, X2, X3, X4, X5, n_epochs = 5000, learning_rate = .001, params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0]), Y=Y)

Epoch 1, Loss 6.051744
MSE:  6.051743994927176
 
Epoch 500, Loss 0.682218
MSE:  0.6822175801261264
 
Epoch 1000, Loss 0.838317
MSE:  0.8383167521140714
 
Epoch 1500, Loss 1.851686
MSE:  1.85168604510917
 
Epoch 2000, Loss 5.559931
MSE:  5.559930889833036
 
Epoch 2500, Loss 19.148895
MSE:  19.148894941728997
 
Epoch 3000, Loss 69.158204
MSE:  69.15820447671763
 
Epoch 3500, Loss 253.908607
MSE:  253.9086071032863
 
Epoch 4000, Loss 938.764537
MSE:  938.7645367648021
 
Epoch 4500, Loss 3485.088091
MSE:  3485.088091458693
 
Epoch 5000, Loss 12977.234779
MSE:  12977.234779267426
 


tensor([-9.1206e+01,  4.4590e+01, -2.2570e+00, -4.1240e+01,  8.8076e+01,
        -1.4715e-14], dtype=torch.float64)

In [74]:
training_loop(X1, X2, X3, X4, X5, n_epochs = 5000, learning_rate = .0001, params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0]), Y=Y)

Epoch 1, Loss 6.051744
MSE:  6.051743994927176
 
Epoch 500, Loss 4.294270
MSE:  4.294269933465941
 
Epoch 1000, Loss 3.092903
MSE:  3.0929030229953023
 
Epoch 1500, Loss 2.274343
MSE:  2.274343022799119
 
Epoch 2000, Loss 1.717445
MSE:  1.7174445068560595
 
Epoch 2500, Loss 1.339554
MSE:  1.3395535228094204
 
Epoch 3000, Loss 1.084292
MSE:  1.084291744636948
 
Epoch 3500, Loss 0.913221
MSE:  0.9132207411478657
 
Epoch 4000, Loss 0.800154
MSE:  0.800154119106354
 
Epoch 4500, Loss 0.727277
MSE:  0.7272767092816058
 
Epoch 5000, Loss 0.682497
MSE:  0.6824970467181195
 


tensor([2.9704e-01, 4.1780e-01, 3.5090e-01, 1.8098e-01, 4.9755e-01, 1.3421e-16],
       dtype=torch.float64)